In [46]:
import numpy as np
# note1:  NN best
# note2:  XGBoost not working
# note3:  KNN not working
# note4:  KRR not working

In [47]:
X = np.load("../../data/6dof/predictTime_X.npy")
Y = np.load("../../data/6dof/predictTime_Y.npy")

In [48]:
import torch
from torch.optim import Adam
import torch.nn as nn

In [49]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [64]:
def setParams(network:torch.nn.Module, decay:float) -> list:
        ''' function to set weight decay
        '''
        params_dict = dict(network.named_parameters())
        params=[]
        weights=[]

        for key, value in params_dict.items():
            if key[-4:] == 'bias':
                params += [{'params':value,'weight_decay':0.0}]
            else:
                params +=  [{'params': value,'weight_decay':decay}]
        return params
    
class FlexDMPForcing(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(12, 200),
            nn.ReLU(),
            nn.Linear(200, 1000),
            nn.ReLU(),
            nn.Linear(1000, 200),
            nn.ReLU(),
            nn.Linear(200, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )
        
    def forward(self, x):
        return self.net(x)

In [65]:
test_data_num = 1000
index_whole = np.arange(len(X))
np.random.shuffle(index_whole)
train_X, train_Y = X[index_whole[:-test_data_num]], Y[index_whole[:-test_data_num]]
test_X, test_Y = X[index_whole[-test_data_num:]], Y[index_whole[-test_data_num:]]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)
train_X, train_Y = torch.from_numpy(train_X).float().to(device), \
            torch.from_numpy(train_Y).float().to(device)
test_X, test_Y = torch.from_numpy(test_X).float().to(device), \
            torch.from_numpy(test_Y).float().to(device)

In [86]:
mean_X = train_X.mean(dim=0)
std_X = train_X.std(dim=0)
mean_Y = train_Y.mean(dim=0)

train_X -= mean_X
train_X /= std_X
test_X -= mean_X
test_X /= std_X
train_Y -= mean_Y

In [88]:
net = FlexDMPForcing().to(device)
param = setParams(net, decay=1e-2)
optimizer = Adam(params=param, lr=4e-3)
# optimizer = Adam(params=net.parameters(), lr=2e-3)
batch_size = 100
Loss = nn.MSELoss()

In [90]:
net.train()
train_l, vali_l = [], []

for epoch in range(1000):
    index = np.arange(len(train_X))
    np.random.shuffle(index)
    L_t = 0.
    counter = 0
    for i in range(len(train_X)//batch_size):
        optimizer.zero_grad()
        x_i, y_i = \
            train_X[index[i*batch_size:(i+1)*batch_size]], train_Y[index[i*batch_size:(i+1)*batch_size]]
        pred = net(x_i)
        L = Loss(pred, y_i)
        L.backward()
        optimizer.step()
        L_t += L.item()
        counter += 1
    with torch.no_grad():
        net.eval()
        pred = net(test_X) + mean_Y
        VAE_l = torch.abs(pred - test_Y).sum() / len(pred) * 0.004
        L = Loss(pred, test_Y)
        vali_l.append(L.item())
        train_l.append(L_t/counter)
        net.train()
    print("Train: ", L_t/counter, "Vali: ", L.item(), "MAE: ", VAE_l.detach().cpu().numpy())

Train:  31.23986200428695 Vali:  25.12968635559082 MAE:  0.015784651
Train:  28.210014384427517 Vali:  24.581308364868164 MAE:  0.015993053
Train:  25.805591957860713 Vali:  25.604162216186523 MAE:  0.01529839
Train:  24.607316952300586 Vali:  25.633827209472656 MAE:  0.015804656
Train:  22.91066701257829 Vali:  21.393068313598633 MAE:  0.013656735
Train:  21.81012152253295 Vali:  22.42047691345215 MAE:  0.015099708
Train:  21.067576063279624 Vali:  22.52703285217285 MAE:  0.014127751
Train:  20.794980633001533 Vali:  22.22456169128418 MAE:  0.014894594
Train:  20.01604677310093 Vali:  20.40318489074707 MAE:  0.013863576
Train:  19.8650296959088 Vali:  17.61278533935547 MAE:  0.012425714
Train:  19.555043630805805 Vali:  19.696395874023438 MAE:  0.013646353
Train:  19.11981161652709 Vali:  21.996673583984375 MAE:  0.014753267
Train:  18.86585975756748 Vali:  17.334476470947266 MAE:  0.012805433
Train:  18.697630883635377 Vali:  17.70925521850586 MAE:  0.012670355
Train:  18.40853475701

Train:  14.956251355040845 Vali:  14.592655181884766 MAE:  0.010798269
Train:  14.952831877564355 Vali:  14.262982368469238 MAE:  0.01059696
Train:  14.878350644832034 Vali:  14.772324562072754 MAE:  0.011185616
Train:  14.9604886497525 Vali:  12.812385559082031 MAE:  0.010209241
Train:  14.917339402137042 Vali:  13.8076171875 MAE:  0.010620113
Train:  14.972222852020813 Vali:  13.041784286499023 MAE:  0.009859764
Train:  14.948808395262244 Vali:  13.32274055480957 MAE:  0.010289272
Train:  14.928772337659657 Vali:  13.729002952575684 MAE:  0.010151737
Train:  14.957668448523652 Vali:  13.567649841308594 MAE:  0.01050737
Train:  14.965673359521002 Vali:  13.241782188415527 MAE:  0.010331156
Train:  14.956178697064626 Vali:  13.630961418151855 MAE:  0.0104791755
Train:  14.961572508846256 Vali:  14.981582641601562 MAE:  0.011736209
Train:  14.87399783031546 Vali:  13.460371017456055 MAE:  0.010182241
Train:  14.955507223211605 Vali:  13.821677207946777 MAE:  0.010915755
Train:  14.89410

Train:  14.759552844136739 Vali:  13.076629638671875 MAE:  0.0102890665
Train:  14.835786139879295 Vali:  14.011933326721191 MAE:  0.010557778
Train:  14.733066226424073 Vali:  14.132000923156738 MAE:  0.010911316
Train:  14.81041774132269 Vali:  13.158210754394531 MAE:  0.010333358
Train:  14.690664023461101 Vali:  13.643608093261719 MAE:  0.0098504275
Train:  14.841647852753564 Vali:  14.893192291259766 MAE:  0.011599283
Train:  14.781139048226446 Vali:  13.299965858459473 MAE:  0.01038879
Train:  14.786981436846068 Vali:  12.797149658203125 MAE:  0.009928669
Train:  14.810112611852961 Vali:  13.614240646362305 MAE:  0.010437798
Train:  14.768970967025208 Vali:  15.210172653198242 MAE:  0.0113549465
Train:  14.795208508333713 Vali:  14.305242538452148 MAE:  0.011589425
Train:  14.7165175081157 Vali:  14.233635902404785 MAE:  0.011000268
Train:  14.867847624442561 Vali:  13.541285514831543 MAE:  0.0104311295
Train:  14.766703720229993 Vali:  14.758628845214844 MAE:  0.011280352
Train:

KeyboardInterrupt: 

In [80]:
net(test_X) - test_Y

tensor([[ 2.4994e+00],
        [-1.2521e+01],
        [-1.9953e+00],
        [ 3.4601e+00],
        [ 3.7718e-01],
        [ 2.3361e+00],
        [ 2.1915e-01],
        [ 2.1711e+00],
        [ 9.2932e-01],
        [-1.5396e-02],
        [ 2.4771e+00],
        [ 3.9398e+00],
        [ 5.8221e-01],
        [ 6.2283e+00],
        [ 1.8665e+00],
        [ 1.7086e+00],
        [ 5.0164e+00],
        [ 2.7312e+00],
        [-8.5384e-01],
        [-6.5621e+00],
        [ 6.8361e-01],
        [ 3.3615e-02],
        [ 4.0456e+00],
        [-4.2135e+00],
        [ 4.5174e-01],
        [-9.6619e+00],
        [-1.3577e+01],
        [-6.5009e-01],
        [-1.6719e+01],
        [ 1.7739e+00],
        [-4.5154e+00],
        [-1.3977e+00],
        [-6.1819e+00],
        [ 4.2514e+00],
        [ 2.7573e-02],
        [ 2.6320e+00],
        [ 2.4495e+00],
        [ 1.2439e+00],
        [-1.1765e-02],
        [-3.5175e+00],
        [-2.5516e+00],
        [ 3.9020e+00],
        [ 1.8934e+00],
        [ 1